# 问题：向用户推荐电影或节目

修改自：
- [使用 SageMaker、MXNet 和 Gluon 实施推荐系统](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/gluon_recommender_system/gluon_recommender_system.ipynb)
- [使用 MNIST 的因子分解机的简介](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/factorization_machines_mnist/factorization_machines_mnist.ipynb)
- [扩展 Amazon SageMaker 因子分解机算法以预测前 X 条推荐](https://aws.amazon.com/blogs/machine-learning/extending-amazon-sagemaker-factorization-machines-algorithm-to-predict-top-x-recommendations/)

## 业务场景简介

您为一家致力于为用户提供点播视频流式传输服务的初创公司工作。公司希望根据用户的观看历史为其介绍电影/节目推荐。

您的工作是利用机器学习来创建要在用户网站上使用的推荐引擎，以解决部分问题。您有权访问历史用户偏好和他们观看的电影的数据集。您可以使用该数据集训练机器学习模型，以推荐要观看的电影/节目。

## 关于该数据集  
Amazon 客户评论数据集是 1995 年至 2015 年期间 Amazon.com 市场上不同产品的评论的集合。客户评论是 Amazon 最重要的数据类型之一。自公司成立以来，收集和显示评论一直是 Amazon 文化的一部分，并且可以说是创新的重要源头之一。有关该数据集的更多详细信息，请参阅 [Amazon 买家评论数据集](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)。

本练习着重于视频的评论。视频数据集包含来自超过 200 万 Amazon 客户的 16 万数字视频的 1 至 5 星的星级评分。

### 特征

**数据列**

- `marketplace`：两个字母的国家/地区代码（在本例中，全为“US”）
- `customer_id`：随机标识符，可用于汇总单个作者撰写的评论
- `review_id`：评论的唯一 ID
- `product_id`：Amazon 标准识别码 (ASIN)。http://www.amazon.com/dp/&lt;ASIN\> 链接到产品的详情页面。
- `product_parent`：ASIN 的父级。多个 ASIN（同一产品的颜色或格式变体）可以汇总到单个父级。
- `product_title`：产品名称说明
- `product_category`：可用于对评论进行分组的广泛产品类别（在本例中为数字视频）
- `star_rating`：产品评级（1 至 5 星）
- `helpful_votes`：评论的有用票数
- `total_votes`：评论获得的总票数
- `vine`：评论是否是作为 Vine 计划的一部分编写的？
- `verified_purchase`：评论是否是来自经过验证的购买？
- `review_headline`：评论本身的标题
- `review_body`：评论文本
- `review_date`：编写评论的日期


**数据格式**
- 制表符 `\t` 分隔文本文件，不带引号或转义字符
- 每个文件的第一行是标题；1 行对应 1 条记录

### 数据集属性

网站：https://s3.amazonaws.com/amazon-reviews-pds/readme.html

该数据集经过 Amazon 许可提供给您，并要遵守 AWS 数字培训服务协议（可从 https://aws.amazon.com/training/digital-training-agreement 获取）的条款。明确禁止出于完成本实验之外的目的复制、修改、出售、导出或使用该数据集。

## 集体讨论和设计…

…可以通过机器学习解答的问题。

大多数项目的第一步是考虑您要提出的问题，可用数据如何支持该问题，以及您将使用哪种工具（在本例中为机器学习模型）来解答问题。这是非常重要的一步，因为它有助于缩小探索范围并明确您将要使用的特征。

请花一点时间在下面的单元格中写下您对数据集的想法。您可以通过机器学习预测哪些事情？ 为什么从业务/客户角度来看具有相关性？ 阐明为什么您认为这些想法很重要。

In [ ]:
# 在此处写下想法

关于如何处理数据，我们可能会有若干想法，但是目前，所有人都将致力于向特定用户推荐视频。

## 推荐和因子分解机

在许多方面，推荐系统推动了机器学习目前的流行。亚马逊最早的成就之一就是“购买该产品的客户，也购买了…”功能。数百万美元的 Netflix 奖金鼓舞了研究、提高了公众的认识，并激发了许多其他数据科学竞赛。

推荐系统可以利用多种数据源和机器学习算法。大多数将各种非监督、监督和强化的学习技术结合到一个整体框架中。但是，核心部分几乎都是基于用户对相似项目的历史评分以及其他相似用户的行为，预测用户对某一项目的评分（或购买）的模型。为此所需的最小数据集是用户项目评分的历史记录（我们已有）。

您将使用的方法是因子分解机。因子分解机是通用的监督式机器学习算法，适用于分类和回归任务。它是线性模型的扩展，旨在简要（简化）捕获高维稀疏数据集中特征之间的交互。这使其成为处理具有点击预测和项目推荐等功能的数据模式的理想之选。

# 步骤 1：问题定义和数据收集

在开始这个项目时，先在下面用几句话来总结在这个场景中要解决的业务问题和要实现的业务目标。同时说明您想让自己的团队衡量的业务指标。确定这些信息后，请清楚地写出机器学习问题陈述。最后，添加一两条备注，说明问题陈述所对应的机器学习的类型。

#### <span style="color: blue;">项目演示：在项目演示中总结这些详细信息。</span>

### 通读业务场景，然后：

### 1. 确定 ML 是否是适合部署的解决方案以及原因。

In [ ]:
# 在此处写下答案

### 2. 定义业务问题、成功指标和期望的 ML 输出。

In [ ]:
# 在此处写下答案

### 3. 确定您要解决的 ML 问题的类型。

In [ ]:
# 在此处写下答案

### 4. 分析您要使用的数据是否合适。

In [ ]:
# 在此处写下答案

### 设置

我们已经确定了工作重点，现在我们来进行一些设置，以便开始解决问题。

**注意：**该笔记本是在 `ml.m4.xlarge` 笔记本实例上创建和测试的。

首先指定：
- 要用于训练和模型数据的 Amazon Simple Storage Service (Amazon S3) 存储桶和前缀 (?)。它们应与笔记本实例、训练和托管位于同一区域。
– 用于提供数据访问权限以便进行训练和托管的 AWS Identity and Access Management (IAM) 角色 [Amazon 资源名称 (ARN)](https://docs.aws.amazon.com/general/latest/gr/aws-arns-and-namespaces.html)。请参阅文档，了解如何创建这些角色。

**注意：**如果笔记本实例、训练和/或托管需要不止一个角色，请用相应的完整 IAM 角色的 ARN 字符串替换 `get_execution_role（）` 调用。

用实验账户提供的资源名称替换 **`<LabBucketName>`**。

In [ ]:
# 根据您的信息更改存储桶和前缀
bucket = '<LabBucketName>'
prefix = 'sagemaker-fm' 

import sagemaker
role = sagemaker.get_execution_role()

现在，加载该示例笔记本的其余部分所需的一些 Python 库。

In [ ]:
import os, subprocess
import warnings
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.mxnet import MXNet
import boto3
import json
import matplotlib.pyplot as plt
import seaborn as sns

# 添加它可以显示单元中的所有输出，而不仅仅是最后一个
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 忽略警告
warnings.filterwarnings("ignore")

# 步骤 2：数据预处理和可视化 
在数据预处理阶段，您应该利用机会探索数据并将其可视化，以便更好地了解数据。首先，导入必要的库并将数据读入 Pandas dataframe。然后，探索数据。查看数据集的形状，探索要使用的列及其类型（数值型、分类型）。对各项特征执行基本统计，以便了解特征的均值和范围；深入分析目标列并确定其分布。

### 要考虑的具体问题
1. 您可以从对特征执行的基本统计中推断出什么？ 

2. 您可以从目标类的分布中推断出什么？

3. 您从数据探索中还推断出了什么？

#### <span style="color: blue;">项目演示：在项目演示中总结这些问题和其他类似问题的答案。</span>

首先将 Amazon S3 公有存储桶中的数据集引入这个笔记本环境。

In [ ]:
# 检查文件是否已位于所需路径中，或者是否需要下载

base_path = '/home/ec2-user/SageMaker/project/data/AmazonReviews'
file_path = '/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz'

if not os.path.isfile(base_path + file_path):
    subprocess.run(['mkdir', '-p', base_path])
    subprocess.run(['aws', 's3', 'cp', 's3://amazon-reviews-pds/tsv' + file_path, base_path])
else:
    print('File already downloaded!')

### 读取数据集

将数据读取到 Pandas dataframe 中，以便您知道要处理的内容。

**注意：**读取文件时，您要设置 `error_bad_lines = False`，因为似乎只有极少数记录会造成问题。

**提示：**您可以使用内置的 Python `read_csv` 函数（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)）。您可以将文件路径直接用于带有 `delimiter ='\ t'` 的 Pandas `read_csv`。

例如：`pd.read_csv('filename.tar.gz', delimiter = '\t', error_bad_lines=False)`

In [ ]:
df = # 在此处输入代码

打印数据集的前几行。 

**提示**：使用 `pandas.head(<number>)` 函数打印行。

In [ ]:
# 在此处输入代码

现在，所有列中包含什么信息？

### 数据集剖析

稍微熟悉一下数据，看看有什么可用的特征。

- `marketplace`：两个字母的国家/地区代码（在本例中，全为“US”）
- `customer_id`：随机标识符，可用于汇总单个作者撰写的评论
- `review_id`：评论的唯一 ID
- `product_id`：Amazon 标准识别码 (ASIN)。http://www.amazon.com/dp/&lt;ASIN\> 链接到产品的详情页面。
- `product_parent`：ASIN 的父级。多个 ASIN（同一产品的颜色或格式变体）可以汇总到单个父级。
- `product_title`：产品名称说明
- `product_category`：可用于对评论进行分组的广泛产品类别（在本例中为数字视频）
- `star_rating`：产品评分（1 至 5 星）
- `helpful_votes`：评论的有用票数
- `total_votes`：评论获得的总票数
- `vine`：评论是否是作为 Vine 计划的一部分编写的？
- `verified_purchase`：评论是否是来自经过验证的购买？
- `review_headline`：评论本身的标题
- `review_body`：评论文本
- `review_date`：编写评论的日期

### 分析和理解数据集

#### 探索数据

**提示：**您可以参考 [此处](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html) 以回答下面的问题。

**问题：**数据集中有多少行和列？

检查数据集的大小。 

**提示**：使用 `<dataframe>.shape` 函数检查 dataframe 的大小

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入答案

**问题：**哪些列包含空值以及包含多少个空值？

打印数据集的摘要。 

**提示**：借助使用关键字参数 `null_counts = True` 的 `<dataframe>.info` 函数

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入答案

**问题：**是否有重复的行？ 如果有，有多少？  

**提示**：使用 `dataframe.duplicated()` ([documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated)) 筛选 dataframe，并检查新 dataframe 的长度。

In [ ]:
duplicates = # 在此处输入代码

# 在此处输入代码

In [ ]:
# 在此处输入答案

### 数据预处理

现在是时候确定要使用哪些特征以及如何为模型做准备了。对于本示例，将自己限定到 `customer_id`、`product_id`、`product_title` 和 `star_rating`。在推荐系统中包括其他特征可能是有好处的，但需要进行大量处理（尤其是文本数据），这超出了该笔记本的范围。

缩小该数据集，仅使用提到的列。 

**提示**：通过将列作为列表传递来选择多个列作为 dataframe。例如：`df[['column_name 1'、'column_name 2']]`

In [ ]:
df_reduced = # 在此处输入代码

缩小数据集后，再次检查是否重复。

In [ ]:
duplicates = # 在此处输入代码

# 在此处输入代码

In [ ]:
# 在此处输入答案

**问题：**为什么现在数据集中会有重复？ 缩小数据集后有什么变化？ 查看重复的前 20 行。

**提示**：使用 `pandas.head(<number>)` 函数打印行。

In [ ]:
# 在此处输入代码

**提示：**查看重复 dataframe 的前两个元素，并查询原始 dataframe df 以查看数据的样子。您可以使用 `query` 函数（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)）。

例如：

```
df_eg = pd.DataFrame({
            'A': [1,2,3,4],
            'B': [
        })
df_eg.query('A > 1 &amp; B > 0')
```

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入答案

在继续操作之前，请删除重复行。

**提示**：使用 `~ `操作符选择所有不重复的行。例如：
    
```
df_eg = pd.DataFrame({
            'A': [1,2,3,4],
            'B': [2,0,5,2]
        })
df_eg[~(df_eg['B'] > 0)]
```

In [ ]:
df_reduced = # 在此处输入代码

### 可视化数据集中的某些行
如果您未在上面部分执行此操作，您可以使用下面的空间来进一步可视化某些数据。具体来看一下 `star_rating`、`customer_id` 和 `product_id` 等特征的分布。

**要考虑的特定问题**

1. 在查看了特征的分布之后，这些特征可以在多大程度上帮助模型？ 您可以通过分布情况推断出哪些可能有助于您更好地理解数据的信息？ 

2. 您应该使用所有数据吗？ 您应该使用哪些特征？

3. 哪个月份的用户评分计数最高？

使用下方的单元格可视化您的数据，并回答这些问题以及您可能感兴趣的其他问题。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中总结这些问题以及类似问题的答案。</span>

使用 `sns.barplot`（[文档](https://seaborn.pydata.org/generated/seaborn.barplot.html)）绘制 `star_rating` 密度和分布。

In [ ]:
# 在此处输入代码以计算有具体评分的评论数

sns.barplot(
    x='index',
    y=<CODE>, # 在此处输入代码
    data=_, # Python 中的下划线符号用于存储上一次操作的输出
    palette='GnBu_d'
)

**问题：**哪个月份的用户评分计数最高？  

**提示**：  
1. 使用 `pd.to_datetime` 将 `review_date` 列转换为日期时间列。 
2. 使用 `review_date` 列中的月份。您可以使用 `<column_name>.dt.month` 访问日期时间列的月份。 
3. 借助使用 `idxmax` 的 `groupby` 函数。 

In [ ]:
# 将评论日期转换为日期时间类型。在此处，您将使用原始 dataframe 'df'
df['review_date'] = # 在此处输入代码

# 按月份计算评分数
df.groupby(<CODE>).star_rating.count().reset_index()

# 再次使用条形图绘制 ratings(y) 和 review_date(x)
sns.barplot(x=<CODE>, y=<CODE>, data=_, palette='GnBu_d') # 在此处输入代码

In [ ]:
# 每月使用 Pandas groupby 函数获取星级评分计数
max_month = df.groupby(<CODE>).star_rating.count().idxmax() # 在此处输入代码
print(f'The month with the most reviews is: {max_month}')

In [ ]:
# 在此处输入答案

**奖金问题（可选）：**评论最多和最少的年份分别是哪年？

In [ ]:
# 使用 Pandas groupby 函数并获得星级评分计数
df.groupby(Z<CODE>).star_rating.count().reset_index() # 在此处输入代码

fig = plt.gcf()
fig.set_size_inches(10, 5)

# 使用条形图绘制 star_rating(y) 和 review_date(x)
sns.barplot(x=<CODE>, y=<CODE>, data=_, palette='GnBu_d') # 在此处输入代码

In [ ]:
# 在此处输入答案

### 数据清理

**问题**：每个客户的评论数量和每个视频的评论数量有何不同？ 使用分位数找出答案。

**提示**：对客户和产品 dataframe 使用 `<dataframe>['columns_name'].value_counts()`，并使用 `<dataframe>.quantile(<list>)` 查找关系。

In [ ]:
customers = # 在此处输入代码
products = # 在此处输入代码

quantiles = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25, 0.5,
             0.75, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 0.995,
             0.999, 1]
print('customers\n', <CODE>) # 在此处输入代码
print('products\n', <CODE>) # 在此处输入代码

In [ ]:
# 在此处输入答案

筛选出此长尾。选择对 18 个或更多视频进行了评分的客户以及具有 95 个以上评论的产品。

In [ ]:
customers1 = # 在此处输入代码
products1 = # 在此处输入代码

# 使用 Pandas merge 函数将 customer1 和 products1 与原始 df_reduced 数据集合并。
reduced_df = (
            df_reduced.merge(pd.DataFrame({'customer_id': customers1.index}))
                      .merge(pd.DataFrame({'product_id': products1.index}))
            )# 在此处输入代码

**问题：** `customers1`、`products1` 和 新 dataframe reduced_df 是什么形状？  

**注意**：对此使用 f 字符串：
```
x= 3
print(f'X = {x}')
```

In [ ]:
print(f'Number of users is {<CODE>} and number of items is {<CODE>}.')# 在此处输入代码
print(f'Length of reduced df is {<CODE>}.')# 在此处输入代码

打印 dataframe 的前 5 列。

In [ ]:
# 在此处输入代码

**问题：** `reduced_df` 是否保持相同的评分率？

In [ ]:
reduced_df[<CODE>].value_counts().reset_index()# 在此处输入代码
sns.barplot(x='index', y='star_rating', data=_, palette='GnBu_d')

In [ ]:
# 在此处输入答案

现在，重新创建每个客户和产品计数的客户和产品分布。

**提示**：在 `customer_id` 和 `product_id` 列上使用 `value_counts()` 函数。

In [ ]:
customers = # 在此处输入代码
products = # 在此处输入代码

fig, axs = plt.subplots(1, 2, figsize=(20, 5))
fig.suptitle('Distribution of counts per customer and product')
sns.distplot(customers, kde=False, ax=axs[0], color='teal')
sns.distplot(products, kde=False, ax=axs[1])

接下来，为每个用户和项目编号，指定它们自己的顺序索引。因此，您能够以稀疏格式保存信息，其中顺序索引表示评分矩阵中的行和列。

要创建 `customer_index` 和 `product_index`，请创建一个新 dataframe，其中将 `customer_id` 作为索引值，并为用户和项目编号创建一个顺序计数器/值。完成两个索引的创建后，请使用 Pandas `merge` 函数将 `customer_index` 与 `product_index 合并。

**提示**：使用 `shape` 函数生成客户和产品总数。使用 `np.arange` 生成一个从 0 到客户和产品数量的数值列表。

In [ ]:
customer_index = pd.DataFrame({'customer_id': customers.index,
                               'user': np.arange(<CODE>)}) # 在此处输入代码
product_index = pd.DataFrame({'product_id': products.index,
                              'item': np.arange(<CODE>)}) # 在此处输入代码

reduced_df = reduced_df.merge(<CODE>).merge(<CODE>)# 在此处输入代码
reduced_df.head()

示例答案：
<div class="output_subarea"><div>

<table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right">
      <th></th>
      <th>customer_id</th>
      <th>product_id</th>
      <th>star_rating</th>
      <th>product_title</th>
      <th>用户</th>
      <th>项目</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>11763902</td>
      <td>B00PSLQYWE</td>
      <td>4</td>
      <td>唐顿庄园第 5 季</td>
      <td>3065</td>
      <td>103</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1411480</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>唐顿庄园第 5 季</td>
      <td>130</td>
      <td>103</td>
    </tr>
    <tr>
      <th>2</th>
      <td>35303629</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>唐顿庄园第 5 季</td>
      <td>4683</td>
      <td>103</td>
    </tr>
    <tr>
      <th>3</th>
      <td>21285980</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>唐顿庄园第 5 季</td>
      <td>449</td>
      <td>103</td>
    </tr>
    <tr>
      <th>4</th>
      <td>29260449</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>唐顿庄园第 5 季</td>
      <td>131</td>
      <td>103</td>
    </tr>
  </tbody>
</table>
</div></div>

## <span style="color:red">实验 2 结束</span>

将项目文件保存到本地计算机。按照以下步骤进行操作：

1. 在页面顶部，单击 **File（文件）**菜单。

1. 选择 **Download as（另存为）**，然后单击 **Notebook (.ipynb)**。 

这会将当前的笔记本下载到计算机上默认的下载文件夹中。

# 步骤 3：模型训练和评估

在将数据集从 dataframe 转换为可供机器学习算法使用的格式之前，您必须完成一些预备步骤。对于 Amazon SageMaker，您需要采取以下步骤：

1. 将数据拆分为 `train_data` 和 `test_data`。   
2. 将数据集转换为适合 Amazon SageMaker 训练作业使用的文件格式。可以是 CSV 文件或记录 protobuf。有关更多信息，请参阅 [适用于训练的常见数据格式](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html)。对于这个问题，数据将是稀疏的，因此可以使用 `scipy.sparse.lilmatrix` 函数，然后使用 `sagemaker.amazon.common.write_spmatrix_to_sparse_tensor` 将函数转换为 `RecordIO protobuf` 格式。   
3. 将数据上传到 Amazon S3 存储桶中。如果您之前没有创建过，请参阅 [创建存储桶](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html).   

使用以下单元格完成这些步骤。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中记下您在此阶段做出的关键决定。</span>

### 准备数据

现在可以开始准备将数据集作为模型的输入。每个模型具有不同的输入需求。Amazon SageMaker 中实施的某些算法要求数据采用 recordIO-wrapped protobuf 形式。您将在以下单元格中进行处理。

首先，将数据集分为训练集和测试集。借此，您就可以估算模型对于客户已评分但训练中未包括的视频的准确性。

从创建 `test_df` dataframe 开始。通过在 `customer_id` 上对 dataframe 进行分组并使用 `last` 函数（类似于 `pd.groupby(' ').last()`）来创建 dataframe。

In [ ]:
test_df = reduced_df.groupby(<CODE>).last().reset_index() # 在此处输入代码

要创建训练数据，请从 `reduced_df` dataframe 中删除 `test_df` 中存在的值。

**提示**：将 `reduced_df` dataframe 与 `test_df` 数据集合并，并以 `customer_id` 和 `product_id` 列作为外联接。

In [ ]:
# 在此处输入代码
train_df = reduced_df.merge(<CODE>,
                            on=['customer_id', 'product_id'],
                            how='outer',
                            indicator=True,
                            indicator=True)
train_df = train_df[(train_df['_merge'] == 'left_only')].reset_index()

In [ ]:
test_df.head()

现在，您可以查看数据的一些基本特征，这些特征稍后将帮助您将特征转换为适当的格式以训练模型。

针对测试和训练数据集的长度创建两个变量：`nb_rating_test` 和 `nb_ratings_train`。

In [ ]:
nb_ratings_test = # 在此处输入代码
nb_ratings_train = # 在此处输入代码
print(f" Training Count: {nb_ratings_train}")
print(f" Test Count: {nb_ratings_test}")

### 数据转换

现在，您可以将 Pandas dataframe 转换为稀疏矩阵。这个过程对于训练和测试都是相同的。因子分解机的 Amazon SageMaker 实施采用 recordIO-wrapped protobuf，您今天拥有的数据是磁盘上的 Pandas Dataframe。因此，您要将数据转换为稀疏矩阵，以表达每个用户与每个电影之间的关系。

In [ ]:
from scipy.sparse import lil_matrix

def loadDataset(df, lines, columns, regressor=True):
    """
    Convert the pandas dataframe into a sparse matrix
    
    Args:
        df: DataFrame
        lines: number of rows of the final sparse matrix
        columns: number of columns of final sparse matrix
        regressor: Boolean value to check if using regression
                  or classification
    返回值：
        X: Feature vector
        Y: Label vector
    """
    # 特征在稀疏矩阵中为独热编码
    
    # 使用 scipy.sparse.lil_matrix 创建 float32 类型的特征向量 X
    # 矩阵的大小是 dataframe 的长度和 
    # 行数加上列数变量 
    X = lil_matrix((<CODE>, lines + columns)).astype('float32') # 在此处输入代码
    
    # 标签存储在向量中。实例化空标签向量 Y。
    Y = # 在此处输入代码
    
    line = 0
    
    # 对于 dataframe 中的每一行，将 1 用作项目和产品编号
    for index, row in df.iterrows():
        X[line,row['user']] = 1
        X[line, lines + (row['item'])] = 1
        line += 1

        if regressor:
            # 如果使用回归，请从行变量附加 star_rating
            Y.append(<CODE>) # 在此处输入代码
        else:
            # 用 1 表示 5 星的星级评分，否则从行变量使用 0
            if int(row['star_rating']) >= 5:
                Y.append(<CODE>) # 在此处输入代码
            else:
                Y.append(<CODE>) # 在此处输入代码
            
    # 将列表转换为类型 float32 的 NumPy 数组。     
    Y = np.array(<CODE>).astype('float32') # 在此处输入代码
    
    return X, Y

使用 `loadDataset` 函数以创建训练和测试集。

In [ ]:
print(customers.shape[0],
      products.shape[0],
      customers.shape[0] + products.shape[0])

# 将 loadDataset 函数与 train_df、customers.shape [0] 和 products.shape [0] 结合使用
X_train, Y_train = loadDataset(<CODE>) # 在此处输入代码

# 将 loadDataset 函数与 test_df、customers.shape [0] 和 products.shape [0] 结合使用
X_test, Y_test = loadDataset(<CODE>) # 在此处输入代码

现在，您的数据为稀疏格式，将其另存为 protobuf 格式，然后将其上传到 Amazon S3。这个步骤可能会让人望而却步，但大部分转换工作都是由 Amazon SageMaker Python SDK 处理的（在下面作为 SageMaker 导入）。

In [ ]:
import io 
import sagemaker.amazon.common as smac

def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)


fm_train_data_path = writeDatasetToProtobuf(X_train, bucket, prefix, 'train', "sparse", Y_train)    
fm_test_data_path = writeDatasetToProtobuf(X_test, bucket, prefix, 'test', "sparse", Y_test)  
  
print("Training data S3 path: ", fm_train_data_path)
print("Test data S3 path: ", fm_test_data_path)

您终于完成了数据准备。太棒了！ 如您所见，清理和准备用于建模的数据需要大量时间和精力。每个数据科学项目都是如此，这个步骤对结果有很大的影响。确保您花费足够的时间来理解和准备数据，以便在以后的所有机器学习活动中进行训练！

## 训练模型

现在是时候训练模型了。您将使用 Amazon SageMaker 训练作业来训练模型。Amazon SageMaker 训练作业是创建模型的简便方法，因为您不用实际编写所有训练代码。这部分工作已经以很不错的容器格式为您处理好了。

从笔记本创建训练作业的一般工作流程是将预测器实例化，传递一些超参数，然后以正确的格式传递数据。这就是在下面的单元格发生的事情。

有关 FM 评估程序的更多详细信息，请参阅 [FactorizationMachines](https://sagemaker.readthedocs.io/en/stable/factorization_machines.html)。

有关超参数的更多信息，请参阅 [因子分解机超参数](https://docs.aws.amazon.com/sagemaker/latest/dg/fact-machines-hyperparameters.html)。

**提示**：示例：

```
sess = sagemaker.Session()

pca = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role,
                                    train_instance_count=1,
                                    train_instance_type='ml.c4.xlarge',
                                    output_path=output_location,
                                    sagemaker_session=sess)
                                    
pca.set_hyperparameters(featuer_dim=50000,
                        num_components=10,
                        subtract_mean=True,
                        algorithm_mode='randomized',
                        mini_batch_size=200)
                        
pca.fit({'train': s3_train_data})
```

In [ ]:
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

output_prefix = 's3://' + bucket + '/sagemaker-fm/model'
instance_type= # Enter your code here
batch_size = # Enter your code here

fm = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, "factorization-machines"),
    role,
    train_instance_count=<CODE>, # 在此处输入代码
    train_instance_type=instance_type,
    output_path=output_prefix,
    sagemaker_session=sagemaker.Session()
)

# 使用超参数。For feature_dim use the column length of X_train 
fm.set_hyperparameters(
                        feature_dim=<CODE>, # 在此处输入代码
                        predictor_type='regressor',
                        mini_batch_size=batch_size,
                        num_factors=64,
                        epochs=25,
                        clip_gradient=5.0,
                        rescale_grad=1.0/batch_size
)

fm.fit({'train': ,# 在此处输入代码,
        'test': # 在此处输入代码
       })

**问题：**更改 `batch_size` 和 `epochs` 对最终指标有什么影响？  

In [ ]:
# 在此处输入答案

**问题：**检查模型输出。使用的指标的含义是什么？ 训练集和测试集之间是否有区别？ 如果有，有什么意义？  

In [ ]:
# 在此处输入答案

### 评估

恭喜！ 您已成功启动了 Amazon SageMaker 训练作业。接下来该怎么做呢？ 您需要一种方法来验证模型是否确实在预测共同值。您如何做到这一点？

从计算原始基准开始，来估计模型的表现。最简单的估算是假设每个用户项目评分只是所有评分的平均评分。基本上是说，您拥有一个仅学会输出所有评论的平均值的模型。

**注意：**通过使用每个视频的平均值，您可以做得更好；但是，在这种情况下，这并不重要，因为可以得出相同的结论。

计算 `star_rating` 的平均值以得到 `naive_guess`。然后，通过将测试中的 `star_rating` 的原始猜想进行平方计算，得到一个平均值。

$average(test(star\_rating) - naive\_guess)^2)$

In [ ]:
naive_guess = np.mean(<CODE>) # 在此处输入代码
print(f'Naive MSE:', np.mean((<CODE>)**2)) # 在此处输入代码

现在，为您的测试数据集计算预测。为此，您需要_部署_刚刚训练的模型。

**注意：**这将与上述 CloudWatch 输出密切相关，但由于跳过 "eval_net "函数中的部分小批量，因此可能会略有不同。

将 `<estimator_name>.deploy` 与 `initial_instance_count=1 和 instance_type=ml.c5.xlarge` 一起使用。

In [ ]:
fm_predictor = # 在此处输入代码

现在，您的终端节点是 'InService'，请评估模型在测试集上的表现。将测试集上的表现与训练集上的表现进行比较。

### 要考虑的关键问题：
1. 您的模型在测试集上的表现与训练集上的表现相比如何？ 您可以从这种比较中推断出什么？ 

2. 准确率、查准率和查全率等指标结果之间有没有明显差异？ 如果是，为什么您会看到这些差异？ 

3. 考虑到您的业务状况和目标，您此时最需要考虑的指标是什么？ 为什么？

4. 您认为最重要指标的结果是否足以满足您的业务需求？ 如果不能，那么您在下一次迭代中可能会更改哪些内容（在特征设计部分，下一步是什么）？ 

用下面的单元格来回答这些问题和其他问题。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中记录这些问题以及您可能会在本节中回答的其他类似问题。在项目演示中记录关键的详细信息和您所做的关键决定。</span>

在部署过程中，需要使用您训练并保存在 Amazon S3 上的模型，创建一个指定大小的实例，在本例中为 `ml.c4.xlarge`。为了获得预测，您需要以 JSON 的序列化形式传递数据。您从推理中得到的输出也将采用序列化 JSON 形式，所以您还需要对其进行反序列化以得到预测值。

In [ ]:
# 为预测器创建一个序列化函数
import json
from sagemaker.predictor import json_deserializer

def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

检查您的训练集效果如何。从模型中获得预测的终端节点。

首先，查看单个预测是什么样子。

Amazon SageMaker 模型容器必须在 60 秒内响应请求。在响应 /invocations 之前，模型本身可能有最多 60 秒的处理时间。为此，一次调用 5 个行的 `predict` 函数，然后将这些行添加到列表中。

In [ ]:
# 将 X_train 数据传递给已部署的预测器 
ytrain_p = []
for i in range(0, 1000, 5):
    preds = fm_predictor.predict(<CODE>)<CODE> # 在此处输入代码
    p = [ytrain_p.append(x['score']) for x in preds]

**问题：**现在您已经有了推理，请进行完整性检查。推理中预测的最小值和最大值是多少？ 这些值是否对应训练数据中的最小值和最大值？

In [ ]:
print('The minimum rating predicted is: ', <CODE>, # 在此处输入代码
      'and the maximum is: ', <CODE> # 在此处输入代码
     )

现在，请检查您的测试数据集。

In [ ]:
Y_pred = []
# 在此处输入代码

**问题：**预测中的最小值和最大值如何？ 如果您检查整个分布（直方图），则可获得奖励积分。

In [ ]:
max(Y_pred), min(Y_pred)

In [ ]:
sns.distplot(Y_pred, kde=False, bins=4)

最后，计算出测试集的均方误差，看看与基准相比有多大的改进。

In [ ]:
print('MSE:', <CODE> )# 在此处输入代码

对于推荐系统，主观准确性也很重要。针对随机用户获取一些推荐，看看是否有直观意义。

尝试使用 200 号用户，看看他们观看并给出高分评价的内容。

In [ ]:
reduced_df[<CODE>].sort_values(
    ['star_rating', 'item'], ascending=[False, True]) # 在此处输入代码

如您所见，此用户喜欢看喜剧、浪漫剧和轻松类别的电影，不喜欢戏剧和奇幻电影。我们来看一下您的模型如何预测该用户的电影评分。

In [ ]:
def prepare_predictions(user_id, number_movies, columns):
    # 创建类似于用于训练数据的稀疏矩阵
    X = il_matrix((<CODE>)).astype('float32')# 在此处输入代码
    movie_index_start = columns - number_movies

    # 填写矩阵。每行将是同一用户与所有可能的影片。
    for row in range(number_movies):
        X[row, user_id - 1] = <CODE> # 在此处输入代码
        X[row, movie_index_start + row] = <CODE> # 在此处输入代码

    return X

user_200 = prepare_predictions(200,
                               <CODE> # 在此处输入代码,
                               <CODE> # 在此处输入代码
                              )

现在，创建一个列表，列出该模型预测的用户 200 对所有影片的评分。

In [ ]:
pred_200 = []
for i in range(0, <CODE>):
    preds = fm_predictor.predict(<CODE>)['predictions']
    p = [pred_200.append(x['score']) for x in preds]

现在，遍历并预测用户 200 对目录中每个普通视频的评分，以了解推荐或不推荐的视频。

通过使用 `reduced_df` dataframe 按项目分组来创建一个新的 dataframe `titles`。使用 `product_title` 列并创建另一个 `score` 列，然后将 `pred_200` 中的值添加到其中。

In [ ]:
titles = reduced_df.groupby(<CODE>)[<CODE>].first().reset_index()
titles['score'] = # 在此处输入代码

**问题：**哪些产品分数最高？  

**提示**：使用 `sort_values` 函数对列 `score` 和 `item` 进行排序，并使用参数 `asecnding=[False,True]`

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入答案

**问题：**您可以从用户的高评分和最低评分节目中得出什么结论？ 

In [ ]:
# 在此处输入答案

查看您的推荐是否与其他用户相关。尝试用户 201。执行与对用户 200 执行的相同操作。

In [ ]:
user_201 = prepare_predictions(<CODE>, products.shape[0], customers.shape[0] + products.shape[0])

pred_201 = []
for i in range(0, user_201.shape[0], 5):
    preds = fm_predictor.predict(user_201[i:i+5].toarray())['predictions']
    p = [pred_201.append(x['score']) for x in preds]

In [ ]:
plt.scatter(pred_200, pred_201)
plt.show()

**问题：**从两个用户之间的散点图可以得出什么结论？  

In [ ]:
# 在此处输入答案：

删除您创建的用于推理的终端节点，因为您将不再使用它。

In [ ]:
sagemaker.Session().delete_endpoint(fm_predictor.endpoint)

## <span style="color:red">实验 3 结束</span>

将项目文件保存到本地计算机。按照以下步骤进行操作：

1. 在页面顶部，单击 **File（文件）**菜单。

1. 选择 **Download as（另存为）**，然后单击 **Notebook (.ipynb)**。 

这会将当前的笔记本下载到计算机上默认的下载文件夹中。

# 迭代 II

# 步骤 4：特征设计

现在，您已经完成了一次训练和评估模型的迭代。鉴于模型第一次获得的结果可能不足以解决您的业务问题，您可以对数据进行哪些更改以改进模型的表现？

### 要考虑的关键问题：
1. 改变机器学习问题如何能帮助您的数据集？ 您已尝试使用回归来解决问题；分类是否有帮助？
2. 您需要做什么才能将机器学习问题更改为机器学习分类问题？ 写下新的问题陈述以进行分类。

#### <span style="color: blue;">项目演示：在项目演示中记录您在本节中使用的主要决策和方法，以及在再次评估模型后获得的任何新性能指标。</span>

现在更改训练数据集，以便根据它们获得的评分输出二进制数据。评分为 5 星时，考虑向用户推荐一些东西，并再次以 protobuf 格式保存在 Amazon S3 中。执行以下操作：  

1. 将 `loadDataset` 函数与选项 `regression=False` 一起使用，以创建训练数据集。  
2. 将数据集写为 protobuf 格式。 
3. 使用`predictor_type='binary_classifier` 重新训练模型。  
4. 将模型部署到终端节点并评估模型，类似于之前在测试集上所做的操作。  
5. 使用混淆矩阵检查在测试集上的表现。  

In [ ]:
X_train_class, Y_train_class = # 在此处输入代码
X_test_class, Y_test_class = # 在此处输入代码

In [ ]:
# 将数据集写为 protobuf
fm_train_data_path = writeDatasetToProtobuf(<CODE>) # 在此处输入代码    
fm_test_data_path = writeDatasetToProtobuf(<CODE>) # 在此处输入代码    
  
print("Training data S3 path: ", fm_train_data_path)
print("Test data S3 path: ", fm_test_data_path)

### 示例代码

```
fm_train_data_path = writeDatasetToProtobuf(X_train_class, bucket, prefix, 'train_class', "sparse", Y_train_class)    
fm_test_data_path = writeDatasetToProtobuf(X_test_class, bucket, prefix, 'test_class', "sparse", Y_test_class) 
```

最后，重新训练模型，从回归更改为二元分类。使用与之前训练模型时相同的代码和设置，但更改 `predictor_type ='binary_classifier`。

In [ ]:
# 重新训练模型
# 在此处输入代码

### 示例代码 
```
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

#output_prefix= 's3://<LabBucketName>/sagemaker-fm/model'

output_prefix = 's3://' + bucket + '/sagemaker-fm/model'
instance_type='ml.m4.xlarge'
batch_size = 512

fm = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, "factorization-machines"),
    role,
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=output_prefix,
    sagemaker_session=sagemaker.Session()
)

fm.set_hyperparameters(feature_dim=X_train.shape[1],
                     # predictor_type='regressor',
                       predictor_type='binary_classifier',
                       mini_batch_size=batch_size,
                       num_factors=128,
                       epochs=25,
                       clip_gradient=5.0,
                       rescale_grad=1.0/batch_size
                       )

fm.fit({'train': fm_train_data_path, 'test': fm_test_data_path})
```

评估该新模型的表现。部署模型，确定序列化程序，然后传递测试数据。

In [ ]:
fm_predictor = fm.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

In [ ]:
# 将测试数据传递到分类器并获取所有预测结果
Y_pred = []
for i in range(0, X_test_class.shape[0], 5):
    preds = fm_predictor.predict(X_test_class[i:i+5].toarray())['predictions']
    p = [Y_pred.append(x['score']) for x in preds]

#### 检查结果

要检查分类器的表现，请计算并绘制混淆矩阵。使用 **Scikit-Learn** 中的实现。

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
true = Y_test_class.astype(int)
predicted = [1 if value > 0.5 else 0 for value in Y_pred]
conf_matrix = confusion_matrix(true, predicted)
print(conf_matrix)
sns.heatmap(conf_matrix)

**问题：**您的模型的准确率是多少？  

**提示**：
$$ Accuracy = \frac{TP + TN}{TP + FP + FN + TN} $$

In [ ]:
# 准确率
# 在此处输入代码

**问题：**与将所有事物预测为 1 的原始基准模型相比，您的模型表现如何？

In [ ]:
(reduced_df.star_rating > 4).value_counts() / reduced_df.shape[0] * 100

In [ ]:
# 在此处输入答案：

In [ ]:
# 删除推理终端节点
sagemaker.Session().delete_endpoint(fm_predictor.endpoint)

## 结合能力与 KNN

您看到分类器模型比回归模型的效果更好。现在，看看您是否可以将其重新包装，以适应 k-近邻 (KNN) 模型来预测与客户喜欢的项目最接近的 *k 最近邻* 项目，然后推荐这些项目，而不是预测评分（回归器）或用户是否喜欢一部电影（二元分类）。

首先从 Amazon S3 下载模型。然后，将其重新包装以适应 KNN 模型。

**注意：**确保您使用的内核是 `conda_mxnet_p36`，以便您可以运行下一个单元。

### 下载模型数据

In [ ]:
import mxnet as mx
model_file_name = 'model.tar.gz'
model_full_path = f'{fm.output_path}/{fm.latest_training_job.job_name}/output/{model_file_name}'
print(f'Model Path: {model_full_path}')

# 下载 FM 模型 
os.system('aws s3 cp ' + model_full_path + ' .')

# 提取模型文件以加载到 MXNet
os.system('tar xzvf ' + model_file_name)
os.system('unzip -o model_algo-1')
os.system('mv symbol.json model-symbol.json')
os.system('mv params model-0000.params')

### 提取模型数据以创建项目和用户潜在矩阵

现在，您要在训练因子分解机之后提取代表每个用户和项目的值。训练的结果是两个矩阵，将它们相乘之后可以尽可能准确地表示目标值（零或一）。

用更专业的数学术语表达，因子分解机的模型输出由三个 N 维数组 (ndarrays) 组成。

    V – (N x k) 矩阵，其中：
        k 是潜在空间的维数
        N 是用户和项目的总数
    w – N 维向量
    b – 单个数字：偏差术语

要提取用作特征的这些值，需要首先加载模型。然后，提取三个矩阵中的每个矩阵的值，并构建 `knn_item_matrix`和t`knn_user_matrix` 矩阵。

In [ ]:
# 提取模型数据
m = mx.module.Module.load('./model', 0, False, label_names=['out_label'])
V = m._arg_params['v'].asnumpy()
w = m._arg_params['w1_weight'].asnumpy()
b = m._arg_params['w0_weight'].asnumpy()

nb_users = customers.shape[0]
nb_item = products.shape[0]

# Item latent matrix - concat(V[i], w[i]). 
knn_item_matrix = np.concatenate((V[nb_users:], w[nb_users:]), axis=1)
knn_train_label = np.arange(1,nb_item+1)

# 用户潜矩阵 - concat (V[u], 1) 
ones = np.ones(nb_users).reshape((nb_users, 1))
knn_user_matrix = np.concatenate((V[:nb_users], ones), axis=1)

## 构建 KNN 模型

现在您有了训练数据，可以将其上传到 KNN 模型。与之前一样，您需要将 protobuf IO 格式的数据保存到 Amazon S3、实例化模型并设置超参数。

首先设置路径和评估程序。

In [ ]:
print('KNN train features shape = ', knn_item_matrix.shape)
knn_prefix = 'knn'
train_key = 'train_knn'
knn_output_prefix = f's3://{bucket}/{knn_prefix}/output'
knn_train_data_path = writeDatasetToProtobuf(knn_item_matrix, bucket,
                                             knn_prefix, train_key,
                                             "dense",
                                             knn_train_label)
print(f'Uploaded KNN train data: {knn_train_data_path}')

nb_recommendations = 100

# 设置评估程序
knn = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, "knn"),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=knn_output_prefix,
    sagemaker_session=sagemaker.Session()
)

现在，您将设置超参数。请注意，这种方法对 KNN 使用默认的 `index_type` 参数。这很精确，但用于大型数据集时可能速度很慢。在这种情况下，您可以使用不同的 `index_type` 参数来获得近似但能更快得到的答案。

有关索引类型的更多信息，请参阅 [k-NN 超参数](https://docs.aws.amazon.com/sagemaker/latest/dg/kNN_hyperparameters.html Hyperparameters-NN)。

In [ ]:
knn.set_hyperparameters(feature_dim=knn_item_matrix.shape[1],
                        k=nb_recommendations,
                        index_metric="INNER_PRODUCT",
                        predictor_type='classifier',
                        sample_size=200000)


knn.fit({'train': knn_train_data_path})

现在，您已经有了经过训练的模型，请保存，以便您可以引用它进行批量推理。

In [ ]:
knn_model_name = knn.latest_training_job.job_name
print("created model: ", knn_model_name)

# 保存模型，以便在下一步的批量推理过程中使用
sm = boto3.client(service_name='sagemaker')
primary_container = {
    'Image': knn.image_name,
    'ModelDataUrl': knn.model_data,
}

knn_model = sm.create_model(
        ModelName = knn.latest_training_job.job_name,
        ExecutionRoleArn = knn.role,
        PrimaryContainer = primary_container)
print("saved the model")

## 批量转换

要查看模型所做的预测，您必须创建推理并查看它们是否有意义。您可以像上次一样重复该过程，并一次使用所有可能的项目组合检查一个用户。但是，Amazon SageMaker 提供了一个批量转换作业，您可以用来基于整个数据集进行推理。有关更多信息，请参阅 [使用批量转换获取整个数据集的推理](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)。

在本节中，您将使用批量转换来预测针对所有用户的前 100 条推荐。

In [ ]:
# 将推理数据上传到 S3
knn_batch_data_path = writeDatasetToProtobuf(knn_user_matrix,
                                             bucket,
                                             knn_prefix,
                                             train_key,
                                             "dense")
print ("Batch inference data path: ",knn_batch_data_path)

# 初始化转换器对象
transformer =sagemaker.transformer.Transformer(
    base_transform_job_name="knn",
    model_name=knn_model_name,
    instance_count=1,
    instance_type=instance_type,
    output_path=knn_output_prefix,
    accept="application/jsonlines; verbose=true",
    
)

# 开始转换作业
transformer.transform(knn_batch_data_path,
                      content_type='application/x-recordio-protobuf',
                      split_type='RecordIO')
transformer.wait()

 

现在，您可以自由检查预测。首先下载。

In [ ]:
# 下载预测 
results_file_name = "inference_output"
inference_output_file = "knn/output/train_knn.out"
s3_client = boto3.client('s3')
s3_client.download_file(bucket, inference_output_file, results_file_name)

In [ ]:
# 打开文件并将其加载到内存
with open(results_file_name) as f:
    results = f.readlines() 

结果包含 100 个近邻电影 ID 及其对应的距离。看看 200 号用户的反应如何。

In [ ]:
test_user_idx = 200
u_one_json = json.loads(results[test_user_idx])
recommended_movies = [int(movie_id) for movie_id in u_one_json['labels']]
distances = [round(distance, 4) for distance in u_one_json['distances']]

print(f'Recommended movie Ids for user #{test_user_idx} : {recommended_movies}')

print(f'Movie distances for user #{test_user_idx} : {distances}')

您获得了最接近用户 200 喜好的影片。现在，您可以查看标题。

In [ ]:
titles_200 = reduced_df[reduced_df.item.isin(recommended_movies)].product_title.unique()
titles_200

将它们与用户 200 最爱的电影进行比较。

In [ ]:
reduced_df.query('user==200 &amp; star_rating == 5')

**问题：**您认为这些推荐有意义吗？ 解释有或没有意义的原因。

In [ ]:
# 在此处输入答案：

In [ ]:
np.isin(titles_200, titles.tail(100).product_title.unique()).sum()

**超级奖励问题：**恢复对用户 201 的预测，了解与用户 200 的对比情况。它们是否仍然相关？ 您认为这种方法是对第一个回归器的改进吗？

In [ ]:
# 恢复对用户 201 的预测

test_user_idx = 201
u_one_json = json.loads(results[test_user_idx])
recommended_movies_201 = [int(movie_id) for movie_id in u_one_json['labels']]

In [ ]:
# 将推荐打印出来

titles_201 = reduced_df[reduced_df.item.isin(recommended_movies_201)].product_title.unique()
titles_201

In [ ]:
# 比较两个预测

overlap = np.isin(titles_200, titles_201).sum()
print(f'The recommendations for "user 201" that are present in "user 200" are: {overlap} out of: {len(titles_200)}')

In [ ]:
# 与用户 201 的喜好进行比较

reduced_df.query('user==201 &amp; star_rating == 5')

In [ ]:
test_user_idx = 900
u_one_json = json.loads(results[test_user_idx])
recommended_movies_900 = [int(movie_id) for movie_id in u_one_json['labels']]
titles_900 = reduced_df[reduced_df.item.isin(recommended_movies_201)].product_title.unique()
overlap_900 = np.isin(titles_200, titles_900).sum()
print(f'The recommendations for "user 900" that are present in "user 200" are: {overlap} out of: {len(titles_200)}')
reduced_df.query('user==900 &amp; star_rating == 5')

In [ ]:
# 在此处输入答案：

您可以做很多事情来改进这些模型，例如在评分之外添加特征、尝试不同的特征选择、超参数调整以及更改模型。最复杂的推荐算法是基于深度学习的。这也可以探索。

就是这样！ 现在，您有了一个可以使用的推荐系统，该系统可以告知您用户最喜欢的前 100 部电影。随时优化并使用超参数和数据，看看是否可以创建更好的推荐系统。

## 最后思考

在该笔记本中，您使用了不同的技术来创建一个仅使用 Amazon SageMaker 内置算法的推荐系统。您学习了如何准备不同格式的数据以及如何进行特征设计。您能够识别出受过训练的模型中的问题，并以不同的方式重新构造问题以达到最终结果。

您现在已经知道，训练模型需要很多步骤、准备和验证。这不是一个简化的过程，而是一个迭代的过程。您可以将它视为一个良性循环，其中通常包含以下步骤：

- 定义（业务）问题。
- 将问题视为机器学习问题进行推敲。
- 准备数据，并进行特征设计。
- 训练和评估模型。
- 部署模型（推理）。
- 监控和评估。

每个步骤都有各自的难点，每个步骤相互补充。因此，务必要注意整个管道，而不仅仅是模型训练。
